In [4]:
import numpy as np
import pandas as pd 
from collections import defaultdict, Counter
import nltk
from nltk.util import ngrams

#Sent1 = "je suis là"
#Sent2 = "tu me vois ? je suis là"
#Sent3 = "je suis me cacher derrière un arbre de bois rouge"
#Sent4 = "je vais me cacher derrière la voiture"

#Line1 = list(ngrams(Sent1.split(), 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
#Line2 = list(ngrams(Sent2.split(), 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
#Line3 = list(ngrams(Sent3.split(), 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
#Line4 = list(ngrams(Sent4.split(), 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))

#liste = Line1+Line2+Line3+Line4

def defineFreq(bigram):
    SetOfWords = list()
    count = 0
    indices = [i for i, x in enumerate(liste) if x == bigram]
    for value in indices:
        if value < len(liste) - 1:
            nextWord = liste[value+1][1]
            SetOfWords.append(nextWord)
            count += 1 
    return [count, dict(Counter(SetOfWords))]
    

def setBigramToString(bigram):
    return bigram[0] +  " " + bigram[1]

def setStringToBigram(string):
    return tuple(string.split())
    
def firstAppearanceOfBigram(bigram,indexListe):
    indices = [i for i, x in enumerate(liste) if x == bigram]
    if indexListe == indices[0]:
        return True
    else:
        return False

    
newDataFrame = pd.DataFrame()
indexOfList = 0
for big in liste[:-1]:
    if (firstAppearanceOfBigram(big,indexOfList) and big[0]!="<s>" and big[1]!="</s>") :
        returnDictio = defineFreq(big)
        row_newDataFrame = pd.DataFrame(returnDictio[1], index=[setBigramToString(big)])
        row_newDataFrame = np.true_divide(row_newDataFrame, returnDictio[0])
        newDataFrame=pd.concat([row_newDataFrame, newDataFrame])
    indexOfList += 1

newDataFrame = newDataFrame.fillna(0)
newDataFrame.index.name = 'bigrams'
print(newDataFrame)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
Sentences=[]
allSentences = []

def generateNextBigram(bigram):
    listOfBigrams=[]
    bigramStr = setBigramToString(bigram)
    for ite in newDataFrame.loc[[bigramStr]].items():
        if ite[1][0] != 0:
            nextBigram = (bigram[1],ite[0])
            listOfBigrams.append(nextBigram)
    return listOfBigrams

def generateNextWord(words):
    listOfWords=[]
    sent = words.split()
    bigr = (sent[-2],sent[-1])
    nextBigr = generateNextBigram(bigr)
    for bi in nextBigr:
        sentence = words+ " " + bi[1]
        listOfWords.append(sentence)
    return listOfWords


def generateSentences(listOfSentences,bigram):
    if not listOfSentences:
        words = setBigramToString(bigram)
        listOfSentences = generateNextWord(words)
        nextbigr = generateNextBigram(bigram)
        generateSentences(listOfSentences,bigram)
        return listOfSentences
    else: 
        for sent in listOfSentences:
            splittedSent = sent.split()
            bigr = (splittedSent[-2],splittedSent[-1])
            nextbigr = generateNextBigram(bigr)
            if nextbigr[0][1]=="</s>":
                if sent not in allSentences:
                    allSentences.append(sent)
            while nextbigr[0][1]!="</s>":
                sentBis = generateNextWord(sent)
                for sentB in sentBis:
                    listOfSentences.append(sentB)
                listOfSentences.remove(sent)
                generateSentences(listOfSentences,bigram)
                return listOfSentences
    

#generateSentences(Sentences,('je', 'suis'))

print('allSentences')
print(allSentences) 

In [6]:
def splitInTrigrams(sentences):
    finalTrig = []
    
    for sentence in sentences:
        finalProba = 1
        trigrams = list(ngrams(sentence.split(), 3, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
        for tri in trigrams:
            if not('<s>' in tri or '</s>' in tri):
                finalProba *= defineProba(tri)
                print(defineProba(tri))
        finalTrig.append((sentence,finalProba))
    return finalTrig

def getBigramFromTrigram(trigram):
    return (trigram[0],trigram[1])
    
def getLastTokenFromTrigram(trigram):
    return trigram[2]
    
def defineProba(trigram):
    proba = 0
    bigramStr = setBigramToString(getBigramFromTrigram(trigram))
    for ite in newDataFrame.loc[[bigramStr]].items():
        if ite[0] == getLastTokenFromTrigram(trigram):
            proba = ite[1][0]
    return proba

def defineAllProbas(listOfTrigrams):
    probs = []
    for trig in listOfTrigrams:
        probs.append(defineProba(trig))
    return probs    
    
        
arrayOfTrigrams = splitInTrigrams(allSentences) 
print(arrayOfTrigrams)
#defineAllProbas(arrayOfTrigrams)    
    